https://docs.python.org/3/howto/descriptor.html#descriptor-example

# 描述符

## 简介

描述符指的是对象的一个属性，对此对象的该属性的访问需要通过重写了的给定方法，类似于`setter()`与`getter()`方法，这些方法包括`__get__()`、`__set__()`与`__delete__()`，而重写了其中任意一种方法的对象即为描述符。

描述符是一种功能强大、应用广泛的接口方式，是实现`property`、静态方法、类方法以及`super()`函数等功能的基础。


## 描述符协议

    descr.__get__(self, obj, type=None) --> value

    descr.__set__(self, obj, value) --> None

    descr.__delete__(self, obj) --> None

重写了以上任意方法的描述符一般可作为对象的一个属性，同时通过重写了的方法访问该属性。

如果一个描述符同时重写了`__get__()`和`__set__()`方法，则被称为数据型描述符。

如果一个描述符只重写了`__get__()`方法，则被称为非数据型描述符。

数据型描述符与非数据型描述符的区别体现在，对于对象`__dict__`中包含的与描述符同名的属性名，访问对象属性时所调用的属性访问函数的优先级。对于访问同名的属性，数据型描述符优先级高于`__dict__`，而`__dict__`优先级高于非数据型描述符。

同时定义`__get__()`与`__set__()`方法，同时令`__set__()`方法抛出异常，则能够实现一个只读的数据型描述符。

**描述符的实例一定是类属性，而不是对象属性。**

## 描述符调用

我们可以直接通过`d.__get__(obj)`显式访问对象的描述符，但更常见的方式是通过`obj.d`隐式访问对象的描述符。

描述符的调用方式取决于`obj`是一个对象还是一个类。

当`b`为对象时，`object.__getattribute__()`会将`b.x`转换为`type(b).__dict__['x'].__get__(b, type(b))`。

当`B`为类时，`type.__getattribute__()`会将`B.x`转换为`B.__dict__['x'].__get__(None, B)`。

这属性访问机制遵循一个访问优先级策略，即数据型描述符的优先级高于实例变量，而实例变量的优先级又高于非数据型描述符，同时`__getattr__()`的优先级最低。

`__getattribute__()`方法实现原理如下。

In [ ]:
def __getattribute__(self, key):
    v = object.__getattribute__(self, key)
    if hasattr(v, '__get__'):
        return v.__get__(None, self)
    return v

- 描述符通过`__getattribute__()`方法调用
- 重写`__getattribute__()`方法将阻止自发的描述符调用
- `object.__getattribute__()`与`type.__getattribute__()`对`__get__()`的调用方式是不同的
- 数据型描述符优先级高于实例字典访问，而实例字典访问优先级又高于非数据型描述符

`super()`函数返回的对象也具有一个用于调用描述符的`__getattribute__()`方法，`super(B, obj).m()`将返回`A.__dict__['m'].__get__(obj, B)`，其中`A`是`B`在mro中的直接父类。



描述符通常用于改变默认的属性访问方式，需要注意的是**描述符的实例一定是类属性，而不是对象属性**。

`descriptor.__get__(self, obj, type=None)`，`descriptor.__set__(self, obj, value)`与`descriptor.__delete__(self, obj)`中的参数`obj`为实例对象，而参数`type`为类。

### 描述符实例

In [1]:
class Desriptor:
    def __init__(self, value):
        self.value = value
    
    def __get__(self, instance, owner):
        print('__get__() is called.', instance, owner)
        return self.value
    
    def __set__(self, instance, value):
        print('__set__() is called.', instance, value)
        self.value = value
        
    def __delete__(self, instance):
        print('__delete__() is called.', instance)

class Widget:
    d = Desriptor(10)
    
w = Widget()

In [2]:
# __set__()
w.d = 1

__set__() is called. <__main__.Widget object at 0x031AB1D0> 1


In [3]:
# __get__()
w.d

__get__() is called. <__main__.Widget object at 0x031AB1D0> <class '__main__.Widget'>


1

In [4]:
# __get__()
Widget.d

__get__() is called. None <class '__main__.Widget'>


1

此时`w.d`会通过`__getattribute__()`方法被转换为`__get__(d, w, Widget)`，而`Widget.d`会通过`__getattribute__()`方法被转换为`__get__(d, None, Widget)`。

In [5]:
# __delete__()
del w.d

__delete__() is called. <__main__.Widget object at 0x031AB1D0>


### 描述符应用场景

property就是基于数据型描述符实现的。但与描述符相比，property不具有复用性，对于同一种类型的属性，在每一类中都需要提供重复的property方法定义。



## property

`property()`函数是一种创建数据型描述符的简单方式，返回一个property属性。

    property(fget=None, fset=None, fdel=None, doc=None)

In [ ]:
class C:
    def getx(self):
        return self.__x
    
    def setx(self, value):
        self.__x = value
        
    def delx(self):
        del self.__x
        
    x = property(getx, setx, delx, "x prorpety")

`property()`函数的本质是将property属性变量封装在一个描述符中，分别在由`__get__()`、`__set__()`与`__delete__()`函数中调用`fget`、`fset`与`fdel`函数。

## python cookbook chapter 8.8

In [40]:
class Person:
    def __init__(self, name):
        # 在执行self.name = name时即调用property.setter方法
        self.name = name
        
    @property
    def name(self):
        print("Person getting name.")
        return self._name
    
    @name.setter
    def name(self, value):
        print("Person setting name.")
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        # name属性的值实际上是存储在_name属性中
        self._name = value
        
    @name.deleter
    def name(self):
        print("Person deleting name.")
        raise AttributeError("Can't delete attribute")
        
p = Person('abc')
print('p.name ==> ', p.name)
print('Person.name ==> ', Person.name)
print('p._name ==> ', p._name)

Person setting name.
Person getting name.
p.name ==>  abc
Person.name ==>  <property object at 0x0345B4E0>
p._name ==>  abc


In [39]:
'''
super(SubPerson, SubPerson)返回的是Person类。
super(SubPerson, self)返回的是一个Person实例。
property本质是一个描述符，而描述符只能是类属性，而不是实例属性，因此name属性只能
通过Person类访问。
'''
class SubPerson(Person):
    @property
    def name(self):
        print('SubPerson getting name.')
        return super().name
    
    @name.setter
    def name(self, value):
        print('SubPerson setting name.')
        super(SubPerson, SubPerson).name.__set__(self, value)
        
    @name.deleter
    def name(self):
        print('SubPerson deleting name.')
        super(SubPerson, SubPerson).name.__delete__(self)
        
s = SubPerson('a')
print('super(SubPerson, SubPerson).name ==> ', super(SubPerson, SubPerson).name)
print('super(SubPerson, s).name ==> ', super(SubPerson, s).name)

SubPerson setting name.
Person setting name.
super(SubPerson, SubPerson).name ==>  <property object at 0x0345BD20>
Person getting name.
super(SubPerson, s).name ==>  a
